# Preparation

In [48]:
from google.colab import userdata
llm_api_key = userdata.get('grok_api_key')

In [49]:
pip install groq

In [50]:
import os

from groq import Groq

client = Groq(
    api_key = llm_api_key,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are revolutionizing the field of natural language processing (NLP) and have numerous applications across various industries. The importance of fast language models can be understood from several perspectives:

1. **Efficient Processing**: Fast language models can process large volumes of text data quickly, which is essential for applications that require real-time processing, such as language translation, sentiment analysis, and chatbots. This enables businesses to respond promptly to customer inquiries, improving customer experience and reducing response times.
2. **Scalability**: Fast language models can handle massive amounts of data, making them ideal for large-scale applications, such as text analysis, information retrieval, and language generation. This scalability enables organizations to analyze vast amounts of data, gain insights, and make informed decisions.
3. **Real-time Applications**: Fast language models are crucial for real-time applications, such a

# Implementation

In [51]:
class Agent:

  def __init__(self, client, system):
    self.client = client
    self.system = system
    self.messages = []
    if self.system is not None:
      self.messages.append({"role": "system", "content": self.system})


  def __call__(self, message = ""):
    if message is not None:
      self.messages.append({"role": "user", "content": message})

    result = self.execute()

    self.messages.append({"role": "assistant", "content": result})
    return result


  def execute(self):
    completion = self.client.chat.completions.create(
      messages=self.messages,
      model="llama-3.3-70b-versatile",
    )
    return completion.choices[0].message.content

In [52]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

In [53]:
# Tools

def calculate (operation):
  return eval(operation)


def get_planet_mass(planet) -> float:
  if planet == "earth":
    return 5.972e24
  elif planet == "mars":
    return 0.642e24
  elif planet == "jupiter":
    return 1.899e27
  elif planet == "saturn":
    return 5.688e26
  elif planet == "uranus":
    return 8.686e25
  elif planet == "neptune":
    return 1.024e26
  elif planet == "mercury":
    return 0.33e24
  elif planet == "venus":
    return 4.87e24
  else:
    return 0.0


In [54]:
neil_tyson = Agent(client, system_prompt)

neil_tyson.messages


[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [55]:
result = neil_tyson("What is the mass of the earth multiplied by 5?")

print(result)

Thought: To find the mass of the Earth multiplied by 5, I first need to determine the mass of the Earth. 

Action: get_planet_mass: Earth
PAUSE


In [56]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [57]:
observation = get_planet_mass("earth")

next_message = "Observation: " + str(observation)

result = neil_tyson(next_message)

print(result)

Thought: Now that I have the mass of the Earth, I can multiply it by 5 to get the final result.

Action: calculate: 5.972e+24 * 5
PAUSE


In [58]:
observation = calculate("5.972e+24 * 5")

next_message = f"Observation: {observation}"

result = neil_tyson(next_message)

print(result)

Thought: I have now calculated the mass of the Earth multiplied by 5, so I can provide the final answer.

Answer: The mass of the Earth multiplied by 5 is 2.986e+25.


In [59]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [60]:
import re

def agent_loop(max_iterations = 10, query = ""):
  agent = Agent(client, system_prompt)
  tools = ["calculate", "get_planet_mass"]
  next_prompt = query
  i = 0
  while i < max_iterations:
      i += 1
      result = agent(next_prompt)
      print(result)

      if "PAUSE" in result and "Action" in result:
          action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
          chosen_tool = action[0][0]
          arg = action[0][1]

          if chosen_tool in tools:
              result_tool = eval(f"{chosen_tool}('{arg}')")
              next_prompt = f"Observation: {result_tool}"

          else:
              next_prompt = "Observation: Tool not found"

          print(next_prompt)
          continue

      if "Answer" in result:
          break


agent_loop(query="What is the mass of Earth plus the mass of Saturn and all of that times 2?")


Thought: To solve this problem, I need to find the mass of Earth and the mass of Saturn, then add them together and multiply the result by 2. First, I need to find the mass of Earth.

Action: get_planet_mass: Earth
PAUSE
Observation: 0.0
Thought: It seems I didn't get the mass of Earth. The observation of 0.0 is likely an error. Let me try again to get the mass of Earth.

Action: get_planet_mass: Earth
PAUSE
Observation: 0.0
Thought: I'm still not getting the correct mass of Earth. This time, I'll try to get the mass of Saturn first, hoping that the function is working correctly for other planets.

Action: get_planet_mass: Saturn
PAUSE
Observation: 0.0
Thought: It seems that the get_planet_mass function is not working correctly, as I'm getting 0.0 for both Earth and Saturn. However, I know that the mass of Earth is approximately 5.972e24 kg and the mass of Saturn is approximately 5.683e26 kg. I can use these values to calculate the result.

Action: calculate: (5.972e24 + 5.683e26) * 2
